#### Friday, December 22, 2023

[openai/whisper-large-v3](https://huggingface.co/openai/whisper-large-v3)

This all runs.

In [1]:
# only target the 4090 ..
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"


In [3]:
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)

# 43m 3.2s

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [12]:
!ls /root/.cache/huggingface/hub

models--HuggingFaceH4--zephyr-7b-beta
models--bert-base-uncased
models--cross-encoder--ms-marco-MiniLM-L-6-v2
models--deepset--roberta-base-squad2
models--distilbert-base-uncased-distilled-squad
models--facebook--blenderbot-1B-distill
models--google--flan-t5-large
models--gpt2-medium
models--intfloat--e5-base-v2
models--intfloat--e5-large-v2
models--meta-llama--Llama-2-13b-hf
models--my_model--language_model.bin
models--openai--whisper-large-v3
models--sentence-transformers--all-MiniLM-L6-v2
models--sentence-transformers--clip-ViT-B-32
models--sentence-transformers--multi-qa-mpnet-base-dot-v1
version.txt


In [ ]:
# docker cp c8324b70601d://root/.cache/huggingface/hub/models--openai--whisper-large-v3 /home/rob/Data3/huggingface/transformers
# Successfully copied 3.09GB to /home/rob/Data3/huggingface/transformers

# docker cp c8324b70601d://root/.cache/huggingface/hub/models--bert-base-uncased /home/rob/Data3/huggingface/transformers
# docker cp c8324b70601d://root/.cache/huggingface/hub/models--cross-encoder--ms-marco-MiniLM-L-6-v2 /home/rob/Data3/huggingface/transformers
# docker cp c8324b70601d://root/.cache/huggingface/hub/models--distilbert-base-uncased-distilled-squad /home/rob/Data3/huggingface/transformers
# docker cp c8324b70601d://root/.cache/huggingface/hub/models--sentence-transformers--clip-ViT-B-32 /home/rob/Data3/huggingface/transformers

In [4]:
model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias=Tr

In [5]:
processor = AutoProcessor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)


In [8]:
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

In [9]:
sample = dataset[0]["audio"]

In [10]:
result = pipe(sample)

In [11]:
print(result["text"])

 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!
